In [1]:
import datetime as dt
import pandas as pd
import copy
import swifter
import sys

In [2]:
class REPORTCONFIG:
    def __init__(self, name, parameter):
        self.name = name
        self.parameters = self.parse_parameter(parameter)
    
    def parse_parameter(self, parameter):
        L = []
        start = False
        for i in range(len(parameter)):
            if parameter[i] == "'" and start == False:
                s = ''
                start = True
                continue
            
            if start:
                if parameter[i] == "'":
                    L.append(s)
                    start = False
                s += parameter[i]
        
        P = dict()
        filter = '+-0123456789[]()&'
        for i in range(0,len(L),2):
            x = ''
            for c in L[i+1]:
                if c in filter:
                    x += c
            try:
                P[L[i]] = int(x)
            except:
                P[L[i]] = x
        return P
    def __str__(self):
        return self.name
    def __repr__(self):
        return self.name + ' ' + str(self.parameters)  

class MEASOBJ:
    def __init__(self, obj, freq):
        self.name = obj
        self.freq = freq
    def __str__(self):
        return f'({self.name}, {self.freq})'
    def __repr__(self):
        return f'({self.name}, {self.freq})'

def parse_measIdToAddMod(s):
    a = s.replace('(','')
    a = a.replace(')','')
    a = a.split('&')
    return (a[0], a[1], a[2])

class data_point: # For ml1 data
    def __init__(self, pd_data): ## Input pd_df.iloc[index]
        self.time = pd_data['time']
        self.pci = pd_data['PCI']
        self.servingcell = pd_data['Serving Cell Index']
        self.earfcn = pd_data['EARFCN']
        self.RSRPQ = float(pd_data['RSRP(dBm)']), float(pd_data['RSRQ(dB)'])
        self.neighbor = self.get_nei_pci_signal_strength(pd_data)

    def get_nei_pci_signal_strength(self, pd_data):
        nei_cells = {}
        for i in range(9, len(pd_data), 3):
            if pd_data[i] == '-':
                break
            else:
                nei_cells[pd_data[i]] = (float(pd_data[i+1]), float(pd_data[i+2]))                
        return nei_cells
    def __repr__(self):
        return ' '.join(['PCI:', self.pci,'|(rsrp, rsrq):', str(self.RSRPQ), '|neighbor:', str(self.neighbor), '|Serving Cell:', self.servingcell, '|EARFCN:', self.earfcn])

class nr_data_point:
    def __init__(self, pd_data): ## Input pd_df.iloc[index]
        self.time = pd_data['time']
        # self.servingcell_index = pd_data['Serving Cell Index']
        self.pci = str(pd_data['Serving Cell PCI'])
        self.arfcn = str(pd_data['Raster ARFCN'])
        self.neighbor = self.get_nei_pci_signal_strength(pd_data)

    def get_nei_pci_signal_strength(self, pd_data):
        nei_cells = {}
        for i in range(6, len(pd_data), 3):
            if pd_data[i] == '-':
                break
            else:
                if i == 6:
                    nei_cells[str(pd_data[i])] = (float(pd_data[i+1]), float(pd_data[i+2]))
                else:
                    nei_cells[pd_data[i]] = (float(pd_data[i+1]), float(pd_data[i+2]))                
        return nei_cells
    def __repr__(self):
        return 'PCI: ' + str(self.pci) + ' nei:' + str(self.neighbor)

# RRC Event Show

In [3]:
## Mobileinsight rrc csv file.
file = '/home/wmnlab/D/database/2023-06-12/Bandlock_8_Schemes_Phone/sm00/#01/data/diag_log_sm00_2023-06-12_16-30-21_rrc.csv'
mi_rrc_df = pd.read_csv(file)
mi_rrc_df["Timestamp"] = mi_rrc_df["Timestamp"].swifter.apply(lambda x: pd.to_datetime(x) + dt.timedelta(hours=8))

In [7]:
original_stdout = sys.stdout

In [8]:
measobj_dict, report_config_dict, measId_dict = {}, {}, {}
nr_measobj_dict, nr_report_config_dict, nr_measId_dict = {}, {}, {}

def reset():
    global measobj_dict, report_config_dict, measId_dict
    global nr_measobj_dict, nr_report_config_dict, nr_measId_dict
    measobj_dict, report_config_dict, measId_dict = {}, {}, {}
    nr_measobj_dict, nr_report_config_dict, nr_measId_dict = {}, {}, {}

# Output file here!
f = open("/home/wmnlab/D/sheng-ru/record.txt", "w")
sys.stdout = f

event_list = [
    "lte-measurementReport", "lte-MeasObjectToAddMod", "lte-ReportConfigToAddMod", 
    "lte-measIdToRemoveList","lte-MeasIdToAddMod", "lte-rrc.t304","rrcConnectionRelease",
    "rrcConnectionSetup","rrcConnectionReestablishmentRequest","rrcConnectionReestablishmentReject",
    "SCellToAddMod-r10", "sCellToReleaseList-r10", "nr-rrc.t304","nr-MeasObjectToAddMod", 
    "nr-ReportConfigToAddMod", "nr-MeasIdToAddMod", "nr-measurementReport",
    "scgFailureInformationNR-r15","nr-Config-r15: release (0)", 
    ]

start = True
RRC_connected = True

for i in range(len(mi_rrc_df)):

    if mi_rrc_df['type_id'].iloc[i] in ["5G_NR_RRC_OTA_Packet", "LTE_RRC_Serv_Cell_Info"]:
        continue

    if start:
        start = False
        print("Initial", "PCI:",mi_rrc_df["PCI"].iloc[0], int(mi_rrc_df["Freq"].iloc[0]))

    for event in event_list:
        if mi_rrc_df[event].iloc[i] not in [0, '0']:
            if event == "SCellToAddMod-r10" and mi_rrc_df["physCellId-r10"].iloc[i] == "nr or cqi report":
                continue
            print('================================================')
            print(mi_rrc_df['Timestamp'].iloc[i])
            break

    if mi_rrc_df["rrcConnectionRelease"].iloc[i] == 1:
        print("Release", "PCI:",mi_rrc_df["PCI"].iloc[i], int(mi_rrc_df["Freq"].iloc[i]))
        RRC_connected = False
        print(f"RRC_connected: {RRC_connected}")
        reset()

    if mi_rrc_df["rrcConnectionSetup"].iloc[i] == 1:
        print("Connection Setup","->", mi_rrc_df["PCI"].iloc[i], int(mi_rrc_df["Freq"].iloc[i]))
        RRC_connected = True
        print(f"RRC_connected: {RRC_connected}")           
    
    if mi_rrc_df["lte-rrc.t304"].iloc[i] == 1:
        pci = mi_rrc_df["PCI"].iloc[i]
        target = int(mi_rrc_df["lte_targetPhysCellId"].iloc[i])
        Freq = int(mi_rrc_df["Freq"].iloc[i])
        dl_CarrierFreq = int(mi_rrc_df["dl-CarrierFreq"].iloc[i])
        print('HO', pci, Freq, '->', target, dl_CarrierFreq)

    if mi_rrc_df["lte-measurementReport"].iloc[i] == 1:
        print("LTE-measurementReport")
        id = str(int(mi_rrc_df['measId'].iloc[i]))
        try:
            x = measId_dict[id]
            event = report_config_dict[x[1]].name
            if 'B1' in event:
                print(id, measId_dict[id], measobj_dict[x[0]], report_config_dict[x[1]], mi_rrc_df['pci-r15'].iloc[i])
            else :
                print(id, measId_dict[id], measobj_dict[x[0]], report_config_dict[x[1]], mi_rrc_df['physCellId'].iloc[i])
            
            print('parameters:', report_config_dict[x[1]].parameters)
        except:
            print(id, "Unknown")

    if mi_rrc_df["nr-measurementReport"].iloc[i] == 1:
        print("NR-measurementReport")
        id = str(int(mi_rrc_df['measId'].iloc[i]))
        try:
            x = nr_measId_dict[id]
            print(id, nr_measId_dict[id], nr_measobj_dict[x[0]], nr_report_config_dict[x[1]], mi_rrc_df['physCellId.1'].iloc[i])
            print('parameters:', nr_report_config_dict[x[1]].parameters)
        except:
            print(id, "Unknown")   

    if mi_rrc_df["lte-MeasObjectToAddMod"].iloc[i] == 1:
        print("MeasObjectToAddMod")
        Id_list = str(mi_rrc_df["measObjectId"].iloc[i]).split('@')
        measobj_list = mi_rrc_df["measObject"].iloc[i].split('@')
        carrierFreq_list = str(mi_rrc_df["carrierFreq"].iloc[i]).split('@')
        carrierFreq_r15_list = str(mi_rrc_df["carrierFreq-r15"].iloc[i]).split('@')
        for a in range(len(Id_list)):
            if measobj_list[a] == "measObjectEUTRA (0)":
                measobj_dict[Id_list[a]] = MEASOBJ(measobj_list[a], carrierFreq_list[0])
                carrierFreq_list.pop(0)
            elif measobj_list[a] == "measObjectNR-r15 (5)":
                measobj_dict[Id_list[a]] = MEASOBJ(measobj_list[a], carrierFreq_r15_list[0])
                carrierFreq_r15_list.pop(0)
        print(measobj_dict)   

    if mi_rrc_df["nr-MeasObjectToAddMod"].iloc[i] == 1:
        print("nr-MeasObjectToAddMod")
        Id_list = mi_rrc_df["measObjectId"].iloc[i].split('@')
        measobj_list = mi_rrc_df["measObject"].iloc[i].split('@')
        ssbFrequency_list = str(int(mi_rrc_df["ssbFrequency"].iloc[i])).split('@')

        for a in range(len(Id_list)):
            if measobj_list[a] == "measObjectNR (0)":
                nr_measobj_dict[Id_list[a]] = MEASOBJ(measobj_list[a], int(ssbFrequency_list[0]))
                ssbFrequency_list.pop(0)     
        print(nr_measobj_dict)
        
    if mi_rrc_df["lte-ReportConfigToAddMod"].iloc[i] == 1:
        print("ReportConfigToAddMod")
        reportConfigId_list = str(mi_rrc_df["lte-reportConfigId"].iloc[i]).split('@')
        eventId_list = mi_rrc_df["lte-eventId"].iloc[i].split('@')
        parameter_list = mi_rrc_df["lte-parameter"].iloc[i].split('@')
        for a in range(len(reportConfigId_list)):
            report_config_dict[reportConfigId_list[a]] = REPORTCONFIG(eventId_list[a], parameter_list[a])
        print(report_config_dict)

    if mi_rrc_df["nr-ReportConfigToAddMod"].iloc[i] == 1: #############
        print("nr-ReportConfigToAddMod")
        reportConfigId_list = mi_rrc_df["nr-reportConfigId"].iloc[i].split('@')
        eventId_list = mi_rrc_df["nr-eventId"].iloc[i].split('@')
        parameter_list = mi_rrc_df["nr-parameter"].iloc[i].split('@')
        for a in range(len(reportConfigId_list)):
            nr_report_config_dict[reportConfigId_list[a]] = REPORTCONFIG(eventId_list[a], parameter_list[a])
        print(nr_report_config_dict)

    if mi_rrc_df["lte-measIdToRemoveList"].iloc[i] != '0':
        print("measIdToRemoveList")
        measIdToRemove_list = str(mi_rrc_df["lte-measIdToRemoveList"].iloc[i]).split('@')
        if len(measIdToRemove_list) == 32:
            measId_dict = {}
        elif len(measId_dict) != 0:
            for a in range(len(measIdToRemove_list)):
                measId_dict.pop(measIdToRemove_list[a])
        print(measId_dict)
    if mi_rrc_df["lte-MeasIdToAddMod"].iloc[i] != '0':
        print("MeasIdToAddMod")
        MeasIdToAdd_list = mi_rrc_df["lte-MeasIdToAddMod"].iloc[i].split('@')
        for a in range(len(MeasIdToAdd_list)):
            x = parse_measIdToAddMod(MeasIdToAdd_list[a])
            measId_dict[x[0]] = (x[1],x[2])
        print(measId_dict)
    if mi_rrc_df["nr-MeasIdToAddMod"].iloc[i] != '0' and mi_rrc_df["nr-MeasIdToAddMod"].iloc[i] != 0:
        print("nr-MeasIdToAddMod")
        MeasIdToAdd_list = mi_rrc_df["nr-MeasIdToAddMod"].iloc[i].split('@')
        for a in range(len(MeasIdToAdd_list)):
            x = parse_measIdToAddMod(MeasIdToAdd_list[a])
            nr_measId_dict[x[0]] = (x[1],x[2])
        print(nr_measId_dict)
    if mi_rrc_df["rrcConnectionReestablishmentRequest"].iloc[i] == 1:
        target = mi_rrc_df["physCellId.3"].iloc[i]
        print(f"rrcConnectionReestablishmentRequest to PCI:{target}")
        measobj_dict, report_config_dict, measId_dict = {}, {}, {}
        nr_measobj_dict, nr_report_config_dict, nr_measId_dict = {}, {}, {}
    if mi_rrc_df["rrcConnectionReestablishmentComplete"].iloc[i] == 1:
        pci = mi_rrc_df["PCI"].iloc[i]
        print(f"successful reestablish to {pci}")
    if mi_rrc_df["rrcConnectionReestablishmentReject"].iloc[i] == 1:
        print(f"rrcConnectionReestablishmentReject")
    if mi_rrc_df["sCellToReleaseList-r10"].iloc[i] != '0' and mi_rrc_df["sCellToReleaseList-r10"].iloc[i] != 0:
        ind = str(mi_rrc_df["SCellIndex-r10"].iloc[i])
        print("Release SCell ID:", ind.split('@'))
    if mi_rrc_df["SCellToAddMod-r10"].iloc[i] == 1 and mi_rrc_df["physCellId-r10"].iloc[i] != "nr or cqi report":
        ind = str(mi_rrc_df["SCellIndex-r10.1"].iloc[i]).split('@')
        pci = mi_rrc_df["physCellId-r10"].iloc[i].split('@')
        dlfreq = str(mi_rrc_df["dl-CarrierFreq-r10"].iloc[i]).split('@')
        S = [(ind[a], pci[a], dlfreq[a]) for a in range(len(ind))]
        for s in S:
            print('Add Scell:', s)
    if mi_rrc_df["nr-rrc.t304"].iloc[i] == 1:
        pci = int(mi_rrc_df["nr_physCellId"].iloc[i])
        if mi_rrc_df["SCellToAddMod-r10"].iloc[i] == 1: 
            print(f'**NR-setup to pci: {pci}.**')
        else:
            print(f'**NR-Ho to pci: {pci}.**')

    if mi_rrc_df["scgFailureInformationNR-r15"].iloc[i]:
        print("NR-SCG Failure.")
        nr_measobj_dict, nr_report_config_dict, nr_measId_dict = {}, {}, {}
    
    if mi_rrc_df["nr-Config-r15: release (0)"].iloc[i]:
        print("NR PSCell release.")
        nr_measobj_dict, nr_report_config_dict, nr_measId_dict = {}, {}, {}

sys.stdout = original_stdout
f.close()
print(f'Finished')

Finished
